In [1]:
#import sys
#print(sys.version)

In [1]:
%pip install ultralytics

^C
Note: you may need to restart the kernel to use updated packages.


In [3]:
import ultralytics
ultralytics.__version__

/Users/carlotacortes/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'8.3.7'

In [4]:
# Not training the model, just using pretrained model trained on COCO dataset. 
# Aquest dataset té som vehicles a part d'altres objectes.
#Fer servir aquest model per la detecció i després track i count
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov5n.pt')

PRO TIP 💡 Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [5]:
#list of COCO dataset classes
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [6]:
#creant una instancia de tracker que ve de from tracker import*
tracker=Tracker()
count=0

In [7]:
#llegir el video
cap=cv2.VideoCapture('/Users/carlotacortes/Desktop/video1min.mp4')

In [8]:
#VARIABLES PER GUARDAR COTXES UP I DOWN
down={}
up={}

counter_down=[]
counter_up=[]

In [9]:
import time

# Initialize the FPS counter
fps = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    # RESOLUCIO IMATGE ORIGINAL: 960X540
    # BAIXAR RESOLUCIO
    frame = cv2.resize(frame, (270, 480))

    # Saltar 1 de cada 2 frames
    if count % 2 != 0:
        continue

    # Convertir el frame a grisos per anar més ràpid: 
    # No es pot utilitzar perquè YOLO treballa en RGB
    # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # YOLO Prediction
    results = model.predict(frame)

    # aconseguint les dades de la bounding box i confiança
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # only work with car class. only count cars. Quedarnos bounding box dels cars
    car_bboxes = []
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            car_bboxes.append([x1, y1, x2, y2])

    # provide les coordinades al tracker
    bbox_id = tracker.update(car_bboxes)

    # draw bbox
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) // 2)
        cy = int((y3 + y4) // 2)

        # Draw bounding box and ID
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
        # cv2.putText(frame, f'ID: {id}', (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Update down/up counters
        red_line_y = 400
        blue_line_y = 450
        offset = 7

        # DOWNWARDS condition: red line first, then blue line
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
            down[id] = cy
        if id in down:
            if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # red dot
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                counter_down.append(id)  # list

        # UPWARDS condition: blue line first, then red line
        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
            up[id] = cy
        if id in up:
            if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # red dot
                cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                counter_up.append(id)  # list

    # Draw lines
    cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # red line
    cv2.line(frame, (0, 450), (850, 450), (255, 0, 0), 3)  # blue line

    # Show counters on the frame
    downwards = len(set(counter_down))  # unique cars counted downwards
    upwards = len(set(counter_up))  # unique cars counted upwards


    # FPS calculation
    fps += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    if elapsed_time > 0:
        current_fps = fps / elapsed_time
        cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


/Users/carlotacortes/Library/Python/3.8/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0: 640x384 3 cars, 1 traffic light, 134.3ms
Speed: 4.9ms preprocess, 134.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)
baixant:  0
pujant:  0

0: 640x384 3 cars, 1 traffic light, 195.6ms
Speed: 3.1ms preprocess, 195.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)
baixant:  0
pujant:  0

0: 640x384 3 cars, 1 traffic light, 115.6ms
Speed: 2.1ms preprocess, 115.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
baixant:  0
pujant:  0

0: 640x384 3 cars, 1 traffic light, 117.8ms
Speed: 3.2ms preprocess, 117.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
baixant:  0
pujant:  0

0: 640x384 3 cars, 1 traffic light, 116.1ms
Speed: 1.6ms preprocess, 116.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)
baixant:  0
pujant:  0

0: 640x384 3 cars, 1 traffic light, 114.8ms
Speed: 2.4ms preprocess, 114.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)
baixant:  0
pujant:  0

0: 6

KeyboardInterrupt: 